In [1]:
# !pip install flash-attn --no-build-isolation

# Ovis

In [2]:
# import torch
# from PIL import Image
# from transformers import AutoModelForCausalLM
# import pandas as pd

# # load data
# df_table_dsc = pd.read_csv('/kaggle/input/sewer-dataset/results/df_table_dsc.csv')
# df_table_prd = pd.read_csv('/kaggle/input/sewer-dataset/results/df_table_prd.csv')
# image_path = '/kaggle/input/sewer-dataset/images'

# # load model
# model = AutoModelForCausalLM.from_pretrained("AIDC-AI/Ovis1.6-Llama3.2-3B",
#                                              torch_dtype=torch.bfloat16,
#                                              multimodal_max_length=8192,
#                                              trust_remote_code=True).cuda()
# text_tokenizer = model.get_text_tokenizer()
# visual_tokenizer = model.get_visual_tokenizer()

# # enter image path and prompt
# image_path = "/kaggle/input/sewer-dataset/images/00002720.png"
# image = Image.open(image_path)
# text = """
# You are a virtual sewer inspection technician equipped to analyze images from CCTV cameras taken inside sewer pipes. Your primary task is to examine each image and classify specific defect types present in the sewer pipeline, as per the provided defect codes. You should carefully evaluate the image and assess which, if any, defects appear.

# For each image, you will:
# 1. Provide a brief, precise description of the scene inside the pipe.
# 2. Identify any defect types observed from the following codes, specifying the code and its severity level based on your assessment.

# The list of defect types and their descriptions:
# - CIW: VA - Water level, measured in percentage.
# - RB: Cracks, breaks, and collapses.
# - OB: Surface damage.
# - PF: Production error.
# - DE: Deformation.
# - FS: Displaced joint.
# - IS: Intruding sealing material.
# - RO: Roots.
# - IN: Infiltration.
# - AF: Settled deposits.
# - BE: Attached deposits.
# - FO: Obstacle.
# - GR: Branch pipe.
# - PH: Chiseled connection.
# - PB: Drilled connection.
# - OS: Lateral reinstatement cuts.
# - OP: Connection with transition profile.
# - OK: Connection with construction changes.

# Provide the output in JSON format as follows:
# - DESCRIPTION: "<Description of the pipe interior and any observed features>"
# - DEFECTS: [
#      { "CODE": "<Defect Code>", "DESCRIPTION": "<Defect Description>", "SEVERITY": "<Assessed Severity Level>" }
# ]

# If no defect is detected, set "CODE" to "NoDefect", with an empty "DESCRIPTION" and "SEVERITY" fields.
# """

# query = f'<image>\n{text}'

# # format conversation
# prompt, input_ids, pixel_values = model.preprocess_inputs(query, [image])
# attention_mask = torch.ne(input_ids, text_tokenizer.pad_token_id)
# input_ids = input_ids.unsqueeze(0).to(device=model.device)
# attention_mask = attention_mask.unsqueeze(0).to(device=model.device)
# pixel_values = [pixel_values.to(dtype=visual_tokenizer.dtype, device=visual_tokenizer.device)]

# # generate output
# with torch.inference_mode():
#     gen_kwargs = dict(
#         max_new_tokens=1024,
#         do_sample=False,
#         top_p=None,
#         top_k=None,
#         temperature=None,
#         repetition_penalty=None,
#         eos_token_id=model.generation_config.eos_token_id,
#         pad_token_id=text_tokenizer.pad_token_id,
#         use_cache=True
#     )
#     output_ids = model.generate(input_ids, pixel_values=pixel_values, attention_mask=attention_mask, **gen_kwargs)[0]
#     output = text_tokenizer.decode(output_ids, skip_special_tokens=True)
#     print(f'Output:\n{output}')


## Issue
It seems cannot support flash attention in this model. The model cannot be used till now. Wait developer to update the code.
https://huggingface.co/AIDC-AI/Ovis1.6-Llama3.2-3B/discussions/2

# Phi-3.5

## For one image

In [3]:
# import torch
# from PIL import Image
# from transformers import AutoModelForCausalLM, AutoProcessor
# import pandas as pd

# # Load data
# df_table_dsc = pd.read_csv('/kaggle/input/sewer-dataset/results/df_table_dsc.csv')
# df_table_prd = pd.read_csv('/kaggle/input/sewer-dataset/results/df_table_prd.csv')
# image_path = '/kaggle/input/sewer-dataset/images/00002720.png'

# # Load model
# model_id = "microsoft/Phi-3.5-vision-instruct"
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     device_map="cuda",
#     trust_remote_code=True,
#     torch_dtype="auto",
#     _attn_implementation='eager'
# )

# # Load processor
# processor = AutoProcessor.from_pretrained(
#     model_id,
#     trust_remote_code=True,
#     num_crops=4  # Using single-frame performance setting as per recommendation
# )

# # Load and prepare image
# image = Image.open(image_path)
# images = [image]
# placeholder = "<|image_1|>\n"

# # Define prompt
# prompt_text = """
# You are a virtual sewer inspection technician equipped to analyze images from CCTV cameras taken inside sewer pipes. Your primary task is to examine each image and classify specific defect types present in the sewer pipeline, as per the provided defect codes. You should carefully evaluate the image and assess which, if any, defects appear.

# For each image, you will:
# 1. Provide a brief, precise description of the scene inside the pipe.
# 2. Identify any defect types observed from the following codes, specifying the code and its severity level based on your assessment.

# The list of defect types and their descriptions:
# - CIW: VA - Water level, measured in percentage.
# - RB: Cracks, breaks, and collapses.
# - OB: Surface damage.
# - PF: Production error.
# - DE: Deformation.
# - FS: Displaced joint.
# - IS: Intruding sealing material.
# - RO: Roots.
# - IN: Infiltration.
# - AF: Settled deposits.
# - BE: Attached deposits.
# - FO: Obstacle.
# - GR: Branch pipe.
# - PH: Chiseled connection.
# - PB: Drilled connection.
# - OS: Lateral reinstatement cuts.
# - OP: Connection with transition profile.
# - OK: Connection with construction changes.

# Provide the output in JSON format as follows:
# - DESCRIPTION: "<Description of the pipe interior and any observed features>"
# - DEFECTS: [
#      { "CODE": "<Defect Code>", "DESCRIPTION": "<Defect Description>", "SEVERITY": "<Assessed Severity Level>" }
# ]

# If no defect is detected, set "CODE" to "NoDefect", with an empty "DESCRIPTION" and "SEVERITY" fields.
# """

# # Apply the processor template for chat
# messages = [
#     {"role": "user", "content": placeholder + prompt_text}
# ]

# prompt = processor.tokenizer.apply_chat_template(
#     messages,
#     tokenize=False,
#     add_generation_prompt=True
# )

# # Prepare inputs for the model
# inputs = processor(prompt, images, return_tensors="pt").to("cuda:0")

# # Generate output
# generation_args = {
#     "max_new_tokens": 1024,
#     "temperature": 0.0,
#     "do_sample": False,
# }

# with torch.inference_mode():
#     generate_ids = model.generate(
#         **inputs,
#         eos_token_id=processor.tokenizer.eos_token_id,
#         **generation_args
#     )

#     # Remove input tokens from output
#     generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
#     response = processor.batch_decode(
#         generate_ids,
#         skip_special_tokens=True,
#         clean_up_tokenization_spaces=False
#     )[0]

# print(f'Output:\n{response}')


In [4]:
# import torch


# torch.cuda.empty_cache()
# torch.cuda.memory_allocated()
# torch.cuda.memory_reserved()

## For all data

In [5]:
import os
import torch
from PIL import Image
from transformers import AutoModelForCausalLM, AutoProcessor
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Load data
df_table_prd = pd.read_csv('/kaggle/input/sewer-dataset/results/df_table_prd.csv')
image_dir = '/kaggle/input/sewer-dataset/images'

# Load model and processor
model_id = "microsoft/Phi-3.5-vision-instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    trust_remote_code=True,
    torch_dtype="auto",
    _attn_implementation='eager'
)
processor = AutoProcessor.from_pretrained(
    model_id,
    trust_remote_code=True,
    num_crops=4  # Setting for single-frame performance
)

# Define the prompt template
prompt_template = """
You are a virtual sewer inspection technician equipped to analyze images from CCTV cameras taken inside sewer pipes. Your primary task is to examine each image and classify specific defect types present in the sewer pipeline, as per the provided defect codes. You should carefully evaluate the image and assess which, if any, defects appear.

For each image, you will:
1. Provide a brief, precise description of the scene inside the pipe.
2. Identify any defect types observed from the following codes, specifying the code and its severity level based on your assessment.

The list of defect types and their descriptions:
- CIW: VA - Water level, measured in percentage.
- RB: Cracks, breaks, and collapses.
- OB: Surface damage.
- PF: Production error.
- DE: Deformation.
- FS: Displaced joint.
- IS: Intruding sealing material.
- RO: Roots.
- IN: Infiltration.
- AF: Settled deposits.
- BE: Attached deposits.
- FO: Obstacle.
- GR: Branch pipe.
- PH: Chiseled connection.
- PB: Drilled connection.
- OS: Lateral reinstatement cuts.
- OP: Connection with transition profile.
- OK: Connection with construction changes.

Provide the output in JSON format as follows:
- DESCRIPTION: "<Description of the pipe interior and any observed features>"
- DEFECTS: [
     { "CODE": "<Defect Code>", "DESCRIPTION": "<Defect Description>", "SEVERITY": "<Assessed Severity Level>" }
]

If no defect is detected, set "CODE" to "NoDefect", with an empty "DESCRIPTION" and "SEVERITY" fields.
"""

# Store predictions and actual labels
predictions = []
actual_labels = []

# Iterate over each row in the ground truth dataframe
for idx, row in df_table_prd.iterrows():
    img_id = row['img_id']
    ground_truth = row['defect_type']
    img_path = os.path.join(image_dir, img_id)
    
    # Check if image file exists
    if not os.path.exists(img_path):
        print(f"Image {img_id} not found. Skipping.")
        continue

    # Load image
    image = Image.open(img_path)
    images = [image]
    placeholder = "<|image_1|>\n"
    
    # Create prompt with image placeholder
    messages = [
        {"role": "user", "content": placeholder + prompt_template}
    ]
    prompt = processor.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Prepare inputs
    inputs = processor(prompt, images, return_tensors="pt").to("cuda:0")

    # Generate output
    generation_args = {
        "max_new_tokens": 1024,
        "temperature": 0.0,
        "do_sample": False,
    }
    with torch.inference_mode():
        generate_ids = model.generate(
            **inputs,
            eos_token_id=processor.tokenizer.eos_token_id,
            **generation_args
        )

        # Process output
        generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
        response = processor.batch_decode(
            generate_ids,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )[0]

    # Extract defect prediction from model output
    if '"CODE": "NoDefect"' in response:
        predicted_defect = "NoDefect"
    else:
        # Extract defect code from JSON output in response
        start_idx = response.find('"CODE": "') + len('"CODE": "')
        end_idx = response.find('"', start_idx)
        predicted_defect = response[start_idx:end_idx]

    # Append results to lists
    predictions.append(predicted_defect)
    actual_labels.append(ground_truth)

    # Optional: print intermediate outputs
    print(f"Image ID: {img_id} | Predicted: {predicted_defect} | Ground Truth: {ground_truth}")

# Calculate evaluation metrics
accuracy = accuracy_score(actual_labels, predictions)
f1 = f1_score(actual_labels, predictions, average='weighted')
report = classification_report(actual_labels, predictions)

print(f"\nAccuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nClassification Report:\n", report)


config.json:   0%|          | 0.00/3.78k [00:00<?, ?B/s]

configuration_phi3_v.py:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-vision-instruct:
- configuration_phi3_v.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3_v.py:   0%|          | 0.00/88.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-vision-instruct:
- modeling_phi3_v.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer.json:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


Image ID: 00617545.png | Predicted: RB | Ground Truth: RB
Image ID: 00635967.png | Predicted: NoDefect | Ground Truth: NoDefect
Image ID: 00002720.png | Predicted: NoDefect | Ground Truth: NoDefect
Image ID: 00497284.png | Predicted: OB | Ground Truth: NoDefect
Image ID: 00231002.png | Predicted: RB | Ground Truth: RB
Image ID: 00468553.png | Predicted: NoDefect | Ground Truth: PF
Image ID: 00953221.png | Predicted: NoDefect | Ground Truth: NoDefect
Image ID: 00242381.png | Predicted: OB | Ground Truth: NoDefect
Image ID: 00292340.png | Predicted: NoDefect | Ground Truth: PF
Image ID: 00184032.png | Predicted: GR | Ground Truth: OB
Image ID: 00544551.png | Predicted: NoDefect | Ground Truth: NoDefect
Image ID: 01051658.png | Predicted: NoDefect | Ground Truth: DE
Image ID: 00841517.png | Predicted: NoDefect | Ground Truth: PF
Image ID: 00306871.png | Predicted: NoDefect | Ground Truth: RO
Image ID: 01057513.png | Predicted: NoDefect | Ground Truth: DE
Image ID: 00852617.png | Predicted

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.

## Parallel

In [6]:
# import os
# import torch
# from PIL import Image
# from transformers import AutoModelForCausalLM, AutoProcessor
# import pandas as pd
# from sklearn.metrics import accuracy_score, f1_score, classification_report

# # Load data
# df_table_prd = pd.read_csv('/kaggle/input/sewer-dataset/results/df_table_prd.csv')
# image_dir = '/kaggle/input/sewer-dataset/images'

# # Load model and processor
# model_id = "microsoft/Phi-3.5-vision-instruct"
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     trust_remote_code=True,
#     torch_dtype="auto",
#     _attn_implementation='eager'
# )

# # Wrap the model in DataParallel
# if torch.cuda.device_count() > 1:
#     model = torch.nn.DataParallel(model)

# model.to("cuda")  # Move model to the first GPU

# processor = AutoProcessor.from_pretrained(
#     model_id,
#     trust_remote_code=True,
#     num_crops=4  # Setting for single-frame performance
# )

# # Define the prompt template
# prompt_template = """
# You are a virtual sewer inspection technician equipped to analyze images from CCTV cameras taken inside sewer pipes. Your primary task is to examine each image and classify specific defect types present in the sewer pipeline, as per the provided defect codes. You should carefully evaluate the image and assess which, if any, defects appear.

# For each image, you will:
# 1. Provide a brief, precise description of the scene inside the pipe.
# 2. Identify any defect types observed from the following codes, specifying the code and its severity level based on your assessment.

# The list of defect types and their descriptions:
# - CIW: VA - Water level, measured in percentage.
# - RB: Cracks, breaks, and collapses.
# - OB: Surface damage.
# - PF: Production error.
# - DE: Deformation.
# - FS: Displaced joint.
# - IS: Intruding sealing material.
# - RO: Roots.
# - IN: Infiltration.
# - AF: Settled deposits.
# - BE: Attached deposits.
# - FO: Obstacle.
# - GR: Branch pipe.
# - PH: Chiseled connection.
# - PB: Drilled connection.
# - OS: Lateral reinstatement cuts.
# - OP: Connection with transition profile.
# - OK: Connection with construction changes.

# Provide the output in JSON format as follows:
# - DESCRIPTION: "<Description of the pipe interior and any observed features>"
# - DEFECTS: [
#      { "CODE": "<Defect Code>", "DESCRIPTION": "<Defect Description>", "SEVERITY": "<Assessed Severity Level>" }
# ]

# If no defect is detected, set "CODE" to "NoDefect", with an empty "DESCRIPTION" and "SEVERITY" fields.
# """

# # Store predictions and actual labels
# predictions = []
# actual_labels = []

# # Iterate over each row in the ground truth dataframe
# for idx, row in df_table_prd.iterrows():
#     img_id = row['img_id']
#     ground_truth = row['defect_type']
#     img_path = os.path.join(image_dir, img_id)
    
#     # Check if image file exists
#     if not os.path.exists(img_path):
#         print(f"Image {img_id} not found. Skipping.")
#         continue

#     # Load image
#     image = Image.open(img_path)
#     images = [image]
#     placeholder = "<|image_1|>\n"
    
#     # Create prompt with image placeholder
#     messages = [
#         {"role": "user", "content": placeholder + prompt_template}
#     ]
#     prompt = processor.tokenizer.apply_chat_template(
#         messages,
#         tokenize=False,
#         add_generation_prompt=True
#     )

#     # Prepare inputs
#     inputs = processor(prompt, images, return_tensors="pt").to("cuda:0")  # Ensure input is on GPU 0

#     # Generate output
#     generation_args = {
#         "max_new_tokens": 1024,
#         "temperature": 0.0,
#         "do_sample": False,
#     }
#     with torch.inference_mode():
#         generate_ids = model.generate(
#             **inputs,
#             eos_token_id=processor.tokenizer.eos_token_id,
#             **generation_args
#         )

#         # Process output
#         generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
#         response = processor.batch_decode(
#             generate_ids,
#             skip_special_tokens=True,
#             clean_up_tokenization_spaces=False
#         )[0]

#     # Extract defect prediction from model output
#     if '"CODE": "NoDefect"' in response:
#         predicted_defect = "NoDefect"
#     else:
#         # Extract defect code from JSON output in response
#         start_idx = response.find('"CODE": "') + len('"CODE": "')
#         end_idx = response.find('"', start_idx)
#         predicted_defect = response[start_idx:end_idx]

#     # Append results to lists
#     predictions.append(predicted_defect)
#     actual_labels.append(ground_truth)

#     # Optional: print intermediate outputs
#     print(f"Image ID: {img_id} | Predicted: {predicted_defect} | Ground Truth: {ground_truth}")

# # Calculate evaluation metrics
# accuracy = accuracy_score(actual_labels, predictions)
# f1 = f1_score(actual_labels, predictions, average='weighted')
# report = classification_report(actual_labels, predictions)

# print(f"\nAccuracy: {accuracy:.4f}")
# print(f"F1 Score: {f1:.4f}")
# print("\nClassification Report:\n", report)
